<a href="https://colab.research.google.com/github/UmbrellaMalware/PythonAnalysisCourse/blob/main/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B0%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Моделирование в анализе данных

В этой работе вам нужно решить задачу машинного обучения на тех данных, которые вы использовали в предыдущей практической работе. Напомним требования к данным.

Подберите на [kaggle](https://www.kaggle.com/datasets/) или в другом источнике датасет, удовлетворяющий следующим требованиям:
- количество строк - не менее 10 тысяч
- количество столбцов - не менее 10, из которых не менее 30% - числовые

К этому моменту у вас уже выполнены этапы понимания и обработки данных. Вам остается их нормализовать (при необходимости) и построить модель, решающую некоторую полезную задачу. Какую именно - зависит от вашего датасета, это может быть классификация, кластеризация или регрессия.

Произведите подбор и настройку параметров модели для решения выбранной вами задачи. Помните, что точность рабочей и пригодной для использования в жизни модели не должна быть ниже 95%, поэтому этапы работы должны быть следующими:

1. Нормализация данных (если необходимо для модели)
2. Выбор и реализация модели для бейзлайна.
3. Если байзлайн дал точность в 95% - попытайтесь улучшить результат. Если не дал - исследуйте существующие алгоритмы для улучшения.
4. Если классические модели машинного обучения из sklearn не показывают хорошую точность, попытайтесь построить нейронную сеть, решающую вашу задачу. Можно использовать torch или keras на ваш выбор.
5. Посла каждого шага оценивайте точность модели разными метриками и визуализируйте результат оценки модели.

Обращайте внимание на стиль и грамотность оформления. Все визуализации должны быть выполнены на Python.

Работу рекомендуется выполнять самостоятельно или в группах по 2 человека.

Данные нельзя менять без уважительной причины. Уважительные причины могут быть следующие:
1. С выбранными данными "ну вообще не построить никакую модель". Решение - согласовать выбор других данных с преподавателем.
2. Вы обзавелись данными к диплому и вы считаете рациональным поработать с ними.
3. Вы работаете с классическими табличными данными, а в дипломе будете делать анализ текста.
В случаях 2 и 3 можно взять новые данные, в прежних группах остаться тоже можно.

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import datetime

In [5]:
! wget https://www.dropbox.com/s/r1c7jnfolfn372l/all_perth_310121.csv

--2021-12-21 13:17:19--  https://www.dropbox.com/s/r1c7jnfolfn372l/all_perth_310121.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:6019:18::a27d:412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/r1c7jnfolfn372l/all_perth_310121.csv [following]
--2021-12-21 13:17:19--  https://www.dropbox.com/s/raw/r1c7jnfolfn372l/all_perth_310121.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc526b3f708ddb8c1580ddf00173.dl.dropboxusercontent.com/cd/0/inline/BcSJ9dJNyiE-lFM0GPeg4fpHnH7C-owm9QLS9_QIHYpECz8WjRw4GL6YbkI60G3J-syXgZJWZpb071c2-MZW5fWGfvWBKW1jWBSwHOvBVn02Ihyl7O1J5KK2IR4M6_2bs75fmqBOi8YSsngHG9G_-Y8h/file# [following]
--2021-12-21 13:17:19--  https://uc526b3f708ddb8c1580ddf00173.dl.dropboxusercontent.com/cd/0/inline/BcSJ9dJNyiE-lFM0GPeg4fpHnH7C-owm9QLS9_QIHYpECz8WjRw4GL6YbkI60G3J-s

In [6]:
rawdata = pd.read_csv('all_perth_310121.csv')

In [7]:
rawdata.GARAGE = rawdata.GARAGE.fillna(0)
rawdata.BUILD_YEAR = rawdata.BUILD_YEAR.fillna(rawdata.BUILD_YEAR.mean())

In [8]:
school = rawdata.drop(["ADDRESS", "SUBURB", "NEAREST_STN", "NEAREST_SCH", "DATE_SOLD", "BUILD_YEAR"], axis=1)
school_nona = school.dropna(axis = 0)
school_predict = school[school.NEAREST_SCH_RANK.isna()]

In [9]:
X = school_nona.drop("NEAREST_SCH_RANK", axis=1)
y = school_nona.NEAREST_SCH_RANK
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [10]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
RFR = RandomForestRegressor()
RFR.fit(X_train, y_train)
RFR.score(X_test,y_test)

0.9732865440955055

In [11]:
cross_val_score(RFR, X,y,cv=5)

array([0.97041324, 0.96584957, 0.96690497, 0.96591224, 0.96701183])

In [12]:
X_pred = school_predict.drop("NEAREST_SCH_RANK", axis=1)
school_predict.drop("NEAREST_SCH_RANK", axis=1)
school_predict["NEAREST_SCH_RANK"]=RFR.predict(X_pred)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [13]:
rawdata.NEAREST_SCH_RANK.fillna(school_predict["NEAREST_SCH_RANK"], inplace=True)

In [14]:
date_sold = pd.to_datetime(rawdata.DATE_SOLD)
rawdata["MONTH_SOLD"] = date_sold.apply(lambda x: x.month)
rawdata["YEAR_SOLD"] = date_sold.apply(lambda x: x.year)

In [15]:
data = rawdata.drop(["ADDRESS", "SUBURB", "NEAREST_SCH", "NEAREST_STN", "DATE_SOLD"], axis=1)

import pandas as pd
from sklearn import preprocessing
y = data.PRICE
X = data.drop("PRICE", axis=1)
x = X.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled)

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [16]:
RFR = RandomForestRegressor()
RFR.fit(X_train, y_train)
RFR.score(X_test,y_test)
y_pred = RFR.predict(X_test)

In [19]:
test = pd.DataFrame(X_test)
test["PREDICTED_PRICE"] = y_pred
test["PRICE"] = y_test
# calculate error in % of price
test["QUALITY"] = np.abs((test["PREDICTED_PRICE"]-test["PRICE"]))/test["PRICE"]*100

In [17]:
from bokeh.plotting import figure
from bokeh.tile_providers import get_provider, WIKIMEDIA, CARTODBPOSITRON, STAMEN_TERRAIN, STAMEN_TONER, ESRI_IMAGERY, OSM
from bokeh.io import output_notebook, show
from bokeh.layouts import row
from bokeh.transform import linear_cmap
from bokeh.palettes import Turbo256, RdYlGn, Inferno256
from bokeh.models import ColorBar, ColumnDataSource, LinearColorMapper, BasicTicker
import math
from bokeh.io import export_png

In [20]:
# Convert long-lat in mercator coordinates
def geographic_to_web_mercator(x_lon, y_lat):     
    if abs(x_lon) <= 180 and abs(y_lat) < 90:          
        num = x_lon * 0.017453292519943295         
        x = 6378137.0 * num         
        a = y_lat * 0.017453292519943295          
        x_mercator = x         
        y_mercator = 3189068.5 * math.log((1.0 + math.sin(a)) / (1.0 - math.sin(a)))         
        return x_mercator, y_mercator      
    else:         
        print('Invalid coordinate values for conversion')      
lons, lats = [], []
for lon, lat in list(zip(test.LONGITUDE, test.LATITUDE)):
    x, y = geographic_to_web_mercator(lon,lat)
    lons.append(x)
    lats.append(y)

In [22]:
# This we will need to have the generated pics in the notebook

cartod = get_provider(CARTODBPOSITRON)
source = ColumnDataSource(dict(x=lons,y=lats))
source.data["color"] = test.QUALITY
fig = figure(plot_width=500, plot_height=500,
           x_range=(12900000, 12916000), y_range=(-3825558, -3692302),
           x_axis_type="mercator", y_axis_type="mercator", toolbar_location=None)
mapper=linear_cmap("color", palette="Inferno256", low=0, high=25)
color_mapper = LinearColorMapper(palette="Inferno256", low=0, high=25)

fig.circle(x="x", y="y", radius=300, alpha=0.5, fill_color=mapper, line_width=0, source=source)

color_bar = ColorBar(color_mapper=color_mapper, ticker= BasicTicker(),
                     location=(0,0), width=10, title="Root Mean Squared Error",
                    major_label_overrides = { 25:">25"})
fig.add_tile(cartod)
fig.add_layout(color_bar, 'right')

show(fig)